In [1]:
import gc
import time
from logzero import logger
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [4]:
ls -lah ../data/

total 24G
drwxrwxr-x 5 ubuntu ubuntu 4.0K Apr 30 07:21 ./
drwxrwxr-x 8 ubuntu ubuntu 4.0K Apr 30 17:30 ../
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 28 16:52 .ipynb_checkpoints/
-rw-r--r-- 1 ubuntu ubuntu 4.8G Apr 30 14:27 click_data.feather
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 29 14:21 kenkoooos/
-rw-r--r-- 1 ubuntu ubuntu  18G May  2 14:38 merge.feather
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 28 17:13 raw/
-rw-rw-r-- 1 ubuntu ubuntu 1.4G Apr 28 17:20 raw.zip


In [5]:
%time
merge = pd.read_feather("../data/merge.feather", nthreads=12)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.44 µs


In [20]:
d = (merge.groupby('channel', as_index=False)
         .count()
         .sort_values('ip', ascending=False)[['channel', 'ip']]
         .rename({'ip':'count_by_channel'})
    )
d.head()

,channel,ip
98,280,10668842
17,107,7337066
183,477,5747279
82,245,5205429
91,265,4710194
